In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
columns=['Ticker','Date','Close','Volume','EPS','B/S','B/P','E/P','S/P','Shares Float','Share Out','Dividend/Share','Shareholder yield','Cashflow','EBITDA/Share','Market,''Market Cap','Momentum 5 days','Momentum 20 days','Momentum 60 days','Momentum 120 days','RSI 14 days','Industry_1','Industry_2','Industry_3']

In [3]:
path=os.path.abspath("/Desktop/rank_stock.csv")

In [4]:
df=pd.read_csv(r"C:\Users\ASUS\Desktop\rank_stock.csv",on_bad_lines = 'warn',parse_dates=True)
df.reset_index(inplace=True)
df=df[df['Industry_1'].notnull()]
df.drop(index=df.index[0], axis=0, inplace=True)
df=df.fillna(value=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16212\2770479244.py:1: DtypeWarning: Columns (13,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\ASUS\Desktop\rank_stock.csv",on_bad_lines = 'warn',parse_dates=True)


In [5]:
df.set_index("index",inplace=True)

In [6]:
df['Cash flow']=pd.to_numeric(df['Cash flow'])

In [7]:
df.columns

Index(['Ticker', 'Date/Time', 'Close', 'Volume', 'EPS', 'B/S', 'B/P', 'E/P',
       'S/P', 'Shares Float', 'Shares Out', 'Dividend Per Share',
       'Shareholder Yield', 'Cash flow', 'EBITDA/Share', 'Market',
       'Market Cap', 'Momentum 5 days', 'Momentum 20 days', 'Momentum 60 days',
       'Momentum 120 days', 'RSI 14 days', 'Industry_1', 'Industry_2',
       'Industry_3'],
      dtype='object')

In [8]:
df['CF/P']=df['Close']/df['Cash flow']
df['EBITDA/EV']=df['EBITDA/Share']/df['Market Cap']

In [9]:
df['Trading Volume']= df['Close']*df['Volume']
df['Av_trading_60']=df['Trading Volume'].rolling(window=60,min_periods=20).mean()
df['Av_trading_20']=df['Trading Volume'].rolling(window=20,min_periods=20).mean()

In [10]:
cols_rank=['EPS','B/S','B/P','E/P','S/P','Shareholder Yield','Market Cap','Momentum 5 days','Momentum 20 days','Momentum 60 days','Momentum 120 days','CF/P','EBITDA/EV']

In [11]:
df['Market_cap']=(df['Shares Float']+df['Shares Out'])*df['Close']
df['Av_marketcap_60']=df['Market Cap'].rolling(window=60,min_periods=20).mean()
df['Av_marketcap_20']=df['Market Cap'].rolling(window=20,min_periods=20).mean()

In [12]:
df_rank=df.groupby(by=['Date/Time'])[cols_rank].rank()

In [13]:
df['valuation_score']=df_rank['B/P']+df_rank['S/P']+df['E/P']+df_rank['Shareholder Yield']+df_rank['CF/P']+df_rank['EBITDA/EV']

In [35]:
df_rank['valuation_score']=df['valuation_score'].rank(pct=True)

In [36]:
df_rank

,EPS,B/S,B/P,E/P,S/P,Shareholder Yield,Market Cap,Momentum 5 days,Momentum 20 days,Momentum 60 days,Momentum 120 days,CF/P,EBITDA/EV,valuation_score
index,,,,,,,,,,,,,,
251,464.5,850.5,946.0,464.5,959.5,600.0,1180.0,658.5,653.0,655.5,655.5,655.5,270.0,0.562519
252,464.5,849.5,969.5,520.5,973.5,600.0,1177.0,660.0,652.5,654.5,655.0,655.0,270.0,0.577024
253,464.5,849.5,965.5,522.5,973.0,600.0,1181.0,665.0,654.5,654.5,655.0,655.0,270.0,0.575253
254,465.5,850.5,974.0,522.0,977.0,600.5,1178.0,662.5,656.0,657.0,655.5,655.5,270.0,0.580722
255,465.5,850.5,975.0,524.5,976.5,600.5,1178.0,250.0,656.0,656.0,655.5,655.5,273.0,0.582050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,565.5,692.0,1164.0,933.5,1154.0,599.0,507.0,203.0,195.0,94.0,101.0,654.0,845.0,0.893354
1048571,565.5,692.0,1188.0,936.5,1172.0,599.0,488.0,51.0,113.0,73.0,71.0,654.0,864.0,0.906446
1048572,565.5,692.0,1173.5,938.5,1158.5,599.0,498.0,88.0,146.0,97.0,68.0,654.0,854.0,0.898627


In [14]:
df_1=df[['Date/Time','Industry_1','Momentum 60 days']]
df_1

,Date/Time,Industry_1,Momentum 60 days
index,,,
251,9/6/2022 0:00,Hoa chat co ban - San pham nhua,0.00
252,9/7/2022 0:00,Hoa chat co ban - San pham nhua,0.00
253,9/8/2022 0:00,Hoa chat co ban - San pham nhua,0.00
254,9/9/2022 0:00,Hoa chat co ban - San pham nhua,0.00
255,9/12/2022 0:00,Hoa chat co ban - San pham nhua,0.00
...,...,...,...
1048570,8/29/2023 0:00,Hoa chat co ban - San pham nhua,-15.63
1048571,8/30/2023 0:00,Hoa chat co ban - San pham nhua,-18.78
1048572,8/31/2023 0:00,Hoa chat co ban - San pham nhua,-15.15


In [15]:
sector_perf=pd.DataFrame(df_1.groupby(by=['Industry_1','Date/Time']).mean()).reset_index().rename(columns={'Momentume_60':'Sector_lv2_perf_60'})

In [16]:
sector_perf=sector_perf.set_index(['Industry_1','Date/Time'])

In [17]:
sector_perf=sector_perf.rank()

In [18]:
sector_perf.reset_index(inplace=True)

In [19]:
df['ta_rating']=(df_rank['Momentum 20 days']+df_rank['Momentum 120 days'])/2

In [37]:
df['Rating']=2/3*(0.7*df['valuation_score']+0.3*df['ta_rating'])+1/3*sector_perf['Momentum 60 days']

In [38]:
from score_stock import penalty_funct

In [39]:
df['Modified_Rating'] = df.apply(lambda row: penalty_funct(row, 'Rating'), axis=1)

In [40]:
df_result=df[(df['Modified_Rating']>=0.7)&(df['Date/Time']=='9/12/2022 0:00')]

In [41]:
df_result

,Ticker,Date/Time,Close,Volume,EPS,B/S,B/P,E/P,S/P,Shares Float,...,Trading Volume,Av_trading_60,Av_trading_20,Market_cap,Av_marketcap_60,Av_marketcap_20,valuation_score,ta_rating,Rating,Modified_Rating
index,,,,,,,,,,,,,,,,,,,,,
255,AAA,9/12/2022 0:00,11.20,1562300.0,0.40,15.94,1.42,0.04,3.57,161133504,...,17497760.0,NaN,NaN,6.086170e+09,NaN,NaN,3480.54,655.75,8117.402000,0.8
1005,AAM,9/12/2022 0:00,11.47,16600.0,1.62,18.20,1.59,0.14,1.77,2890764,...,190402.0,9.307863e+07,7.494672e+07,1.530321e+08,3.929455e+09,2.979806e+09,4620.64,655.75,3402.115333,0.8
1505,A32,9/12/2022 0:00,26.85,100.0,5.39,31.64,1.18,0.20,3.94,2895900,...,2685.0,9.536078e+04,9.681760e+04,2.603349e+08,1.201889e+08,1.369898e+08,4155.20,655.75,5173.910000,0.8
3255,AAT,9/12/2022 0:00,10.09,1296300.0,1.40,11.36,1.13,0.14,1.49,53222056,...,13079667.0,1.692498e+06,5.059553e+06,1.180768e+09,2.447433e+08,3.352655e+08,3450.14,655.75,2441.548667,0.8
4255,AAV,9/12/2022 0:00,8.20,50800.0,0.02,11.72,1.43,0.00,0.88,39719880,...,416560.0,4.833705e+06,3.106374e+06,8.914019e+08,4.011373e+08,4.281098e+08,3046.00,655.75,2404.616667,0.8
4505,ABB,9/12/2022 0:00,10.12,557900.0,1.44,13.87,1.37,0.14,0.95,480127424,...,5645948.0,7.146839e+06,8.095014e+06,1.438127e+10,1.351017e+09,3.200019e+09,2961.14,655.75,1719.682000,0.8
7008,ABS,9/12/2022 0:00,11.27,258700.0,0.34,17.40,1.54,0.03,1.78,49237104,...,2915549.0,1.908178e+07,1.202566e+07,1.456502e+09,7.612919e+09,6.033442e+09,3393.53,655.75,6209.797333,0.8
7258,ABT,9/12/2022 0:00,34.10,300.0,5.46,39.09,1.15,0.16,1.54,1475676,...,10230.0,1.649033e+07,9.396096e+06,4.519250e+08,5.864986e+08,5.415358e+08,4359.16,655.75,9576.424667,0.8
8258,ABC,9/12/2022 0:00,8.65,0.0,38.14,26.25,3.04,4.41,7.87,14617000,...,0.0,2.164095e+04,8.074250e+03,3.028019e+08,3.677926e+08,3.457014e+08,4965.41,655.75,7313.674667,0.8
